In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

from nemo.util import grid_2d
from nemo.plotting import plot_surface, plot_path_3d

%load_ext autoreload
%autoreload 2

g = 9.8  # m/s^2

Dynamics of a Dubin's car moving on a sloped plane (no slip)

- $\theta(t)$ - vehicle heading 
- $\alpha$ - slope angle (always positive)
- $\psi$ - slope direction
- $\omega$ - yaw rate (commanded)
- $a$ - acceleration (commanded)
- $\alpha_{eff}$ - effective slope angle (between -90 and 90, positive is uphill, negative is downhill)

$\theta(t) = \theta_0 + \omega t$ \
$\alpha_{eff}(t) = \alpha \cos(\theta(t) - \psi)$ \
$a_{net}(t) = a - g \sin(\alpha_{eff}(t))$

Full dynamics:
- State: $(x, y, V, \theta)$
    - Height field gives $z=H(x,y)$ and slope angle and direction thru gradient
- control: $(a, \omega)$
    - Fixed between waypoints

Load a height field

In [ ]:
import torch
from nemo.nemo import Nemo
from nemo.util import grid_2d

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

nemo = Nemo()
nemo.load_weights('../models/AirSimMountains/AirSimMountains_encs.pth', '../models/AirSimMountains/AirSimMountains_mlp.pth')

In [ ]:
nemo.plot(N=512, bounds=(-.75, .45, -.6, .6))

In [ ]:
nemo.plot_grads(N=512, clip=2.0)

Normal vector: (dh/dx, dh/dy, 1)


Coordinate frame:

Rover heading: \
+Y: North, $\theta = \pi/2$ \
+X: East, $\theta = 0$

Slope direction angle: \
+Y: $\psi = 0$ \
+X: $\psi = \pi/2$

positive gradient is uphill, negative gradient is downhill

In [ ]:
# Simulate dynamics for (a, omega)
init_state = np.array([0.4, 0., 0., 5*np.pi/6])  # (x, y, v, theta)
u = np.array([1.0, 0.7])     # (a, omega)
dt = 0.01

l = 0.01  # vehicle length
w = 0.005  # vehicle width

N_iters = 300
state = init_state
state_hist = []
log = []

for i in range(N_iters):
    # Unpack state and control
    x, y, v, theta = state
    a, omega = u

    # Query height field to get slope angle and direction
    pos = torch.tensor([[x, y]], device=device, requires_grad=True)
    _, grad = nemo.get_heights_with_grad(pos)
    psi = torch.arctan2(grad[0][1], grad[0][0])  # slope direction (0 is +Y, pi/2 is +X)
    slope = torch.arctan(grad.norm())  # slope angle (positive is uphill, negative is downhill) 
                                        # TODO: from tests below, it seems that NN grad slope is approximately
                                        #       x3 real slope for some reason - address/investigate this
    psi = psi.detach().cpu().numpy()
    slope = slope.detach().cpu().numpy()
    
    # Calculate effective slope (pitch)
    phi_grad = slope * np.cos(theta - psi)
    
    # Get pitch from sampling points
    dl = l/2 * np.array([np.cos(theta), np.sin(theta)])
    points = np.vstack((np.array([x, y]) + dl, np.array([x, y]) - dl))
    pos = torch.tensor(points, device=device)
    z = nemo.get_heights(pos).detach().cpu().numpy().flatten()
    phi = np.arctan2(z[0] - z[1], l)
    
    # Calculate acceleration
    a_net = a - g * np.sin(phi)

    # Integrate velocity and position
    v += a_net * dt
    x += v * np.cos(theta) * np.cos(phi) * dt
    y += v * np.sin(theta) * np.cos(phi) * dt

    # Integrate theta
    theta += omega * dt  # turning rate proportional to velocity

    # Update and log state
    state = np.array([x, y, v, theta])
    state_hist.append(state)
    log.append([phi, phi_grad, a_net])

In [ ]:
state_hist = np.array(state_hist)
xy = state_hist[:, :2]
z = nemo.get_heights(torch.tensor(xy, device=device)).detach().cpu().numpy().flatten()

In [ ]:
log = np.array(log)
plt.figure()
plt.plot(log[:, 0], label='phi')
plt.plot(log[:, 1], label='phi_grad')
plt.legend()
plt.title('Pitch')
plt.show()

In [ ]:
plt.scatter(log[:,0], log[:,1])
plt.axis('equal')
plt.grid(True)

In [ ]:
fig = nemo.plot(N=128, bounds=(-.75, .45, -.6, .6))
fig = plot_path_3d(x=xy[:,0], y=xy[:,1], z=z.flatten(), color=np.arange(len(z.flatten())), fig=fig)
fig.show()

Back out other states and controls from flat outputs $(x(t), y(t))$

In [ ]:
x = xy[:,0]
y = xy[:,1]
xdot = np.hstack((1e-5, np.diff(x)))
ydot = np.hstack((1e-5, np.diff(y)))
xddot = np.hstack((1e-5, np.diff(xdot)))
yddot = np.hstack((1e-5, np.diff(ydot)))
v = np.sqrt(xdot**2 + ydot**2)
theta = np.arctan2(ydot, xdot)

In [ ]:
xy_torch = torch.tensor(xy, dtype=torch.float32).to(device)
xy_torch.requires_grad = True
_, grad = nemo.get_heights_with_grad(xy_torch)
psi = torch.atan2(grad[:,1], grad[:,0]).cpu().detach().numpy()  
alpha = torch.atan(grad.norm(dim=1)).cpu().detach().numpy()
# psi = 0.0
# alpha = 0.1
alpha_eff = alpha * np.cos(theta - psi)
g_eff = g * np.sin(alpha_eff)

In [ ]:
plt.plot(alpha_eff)

In [ ]:
u = np.zeros((len(x), 2))
for i in range(len(x)):
    J = np.array([[np.cos(theta[i]), -v[i]*np.sin(theta[i])],
                  [np.sin(theta[i]), v[i]*np.cos(theta[i])]])
    b = np.array([[xddot[i] + g_eff[i]*np.cos(theta[i])],
                  [yddot[i] + g_eff[i]*np.sin(theta[i])]])
    u[i] = np.linalg.solve(J, b).flatten()

In [ ]:
u

Test slopes

In [ ]:
SLOPE = 1.0

N = 128
bounds = 2*np.array([-1., 1., -1., 1.])

xs = torch.linspace(bounds[0], bounds[1], N, device=device)
ys = torch.linspace(bounds[2], bounds[3], N, device=device)
XY_grid = torch.meshgrid(xs, ys, indexing='xy')
XY_grid = torch.stack(XY_grid, dim=-1)
x_grid = XY_grid[:,:,0].detach().cpu().numpy()
y_grid = XY_grid[:,:,1].detach().cpu().numpy()
xy_data = XY_grid.reshape(-1,2)

zs = torch.linspace(-SLOPE, SLOPE, N, device=device)
z_grid = zs.repeat(N,1)
z_data = z_grid.reshape(-1,1)
z_grid = z_grid.detach().cpu().numpy()

In [ ]:
fig = plot_surface(x=x_grid, y=y_grid, z=z_grid)
fig.show()

In [ ]:
import torch.nn as nn

In [ ]:
nemo = Nemo()
nemo.encoding.to(device)
nemo.height_net.to(device)

In [ ]:
xy_data = xy_data.half()
z_data = z_data.half()

In [ ]:
# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam([{'params': nemo.encoding.parameters()},
                              {'params': nemo.height_net.parameters()}], lr=1e-5)

# Train the network
for step in range(5000):
    # Forward pass
    pred = nemo.get_heights(xy_data)

    # Compute loss
    loss = criterion(pred, z_data)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 500 steps
    if step % 500 == 0:
        print(f"Step {step}, Loss {loss.item()}")

In [ ]:
z_pred = pred.reshape(N, N).detach().cpu().numpy()

fig = plot_surface(x=x_grid, y=y_grid, z=z_pred)
fig.show()

In [ ]:
xy_data.requires_grad = True
z, grad = nemo.get_heights_with_grad(xy_data)

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
x_grad = grad[:,0].reshape(N,N).detach().cpu().numpy()
y_grad = grad[:,1].reshape(N,N).detach().cpu().numpy()

fig = make_subplots(rows=1, cols=2, subplot_titles=('X Gradient', 'Y Gradient'), horizontal_spacing=0.15)
fig.add_trace(go.Heatmap(z=x_grad, colorbar=dict(len=1.05, x=0.44, y=0.5)), row=1, col=1)
fig.add_trace(go.Heatmap(z=y_grad, colorbar=dict(len=1.05, x=1.01, y=0.5)), row=1, col=2)
fig.update_layout(width=1300, height=600, scene_aspectmode='data')
fig.show()

In [ ]:
# Histogram of x_grad
fig = px.histogram(x_grad.flatten())
fig.show()

In [ ]:
fig = px.histogram(y_grad.flatten())
fig.show()